### import required packages

In [1]:
%load_ext lab_black

In [2]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [3]:
import pandas as pd

In [4]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [5]:
from df2gspread import df2gspread as d2g

### datawrapper credentials

In [6]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

### google sheets setup

In [7]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "jupyter-integration.json", scope
)
gc = gspread.authorize(credentials)

In [8]:
spreadsheet_key = "1fJSfICdiVfxUuKS8vczcq0Bn-Ha3fihIcYZIkT7QpMQ"
book = gc.open_by_key(spreadsheet_key)

### ended up publishing google sheet as csv for easier reading

In [9]:
# Opening the worksheet by using Worksheet ID
# workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
# sheet = workbook.worksheet("Incumbents Losing Renomination")
# Pulling the data and transform it to the data frame
# values = sheet.get_all_values()
# df = pd.DataFrame(values[3:], columns=values[2])

In [10]:
# df.rename(
#    columns={"2022 (as of August 2, 2022): 11 House (4 D, 7 R)*": "person"},
#    inplace=True,
# )

In [11]:
df = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ7vCkt60vIsLREGWN0KIz9IRBxFnpOa8zbAc_oEF8unKP0QtfwEFSmaJWrMwtZAW_xAcb9tJ_hUWzg/pub?gid=638845180&single=true&output=csv",
    skiprows=2,
    names=["candidate", "results", "date_defeat", "elected"],
    header=0,
    skipfooter=25,
    parse_dates=["date_defeat"],
    engine="python",
)

### split candidate, party and state into separate columns

In [12]:
df[["candidate", "party"]] = df["candidate"].str.split(",", expand=True, n=1)

# .str.split("-",expand=True).str.split(" (",expand=True)

In [13]:
df[["party", "state"]] = df["party"].str.split("-", expand=True, n=1)

In [14]:
df[["state", "opponent"]] = (
    df["state"].str.replace("\)", "", regex=True).str.split(" \(", expand=True, n=1)
)

### drop results column (and others?)

In [15]:
df = df.drop(columns={"results"})

In [16]:
df

,candidate,date_defeat,elected,party,state,opponent
0,Rep. David McKinley,"May 10, 2022",2010,R,West Virginia,Rep. Alex Mooney
1,Rep. Madison Cawthorn,"May 17, 2022",2020,R,North Carolina,Chuck Edwards
2,Rep. Kurt Schrader,"May 17, 2022",2008,D,Oregon,Jamie McLeod-Skinner
3,Rep. Carolyn Bourdeaux,"May 24, 2022",2020,D,Georgia,Rep. Lucy McBath
4,Rep. Tom Rice,"June 14, 2022",2012,R,South Carolina,Russell Fry
...,...,...,...,...,...,...
227,Rep. Leonard Farbstein,NaN,NaN,D,New York,Bella S. Abzug
228,Rep. Michael Feighan,NaN,NaN,D,Ohio,James V. Stanton
229,Rep. Samuel Friedel,NaN,NaN,D,Maryland,Parren J. Mitchell
230,Rep. Jacob Gilbert,NaN,NaN,D,New York,Rep. James H. Scheuer


### add liz cheney and a trump endorsements column

In [17]:
df = df.append(
    {
        "candidate": "Rep. Liz Cheney",
        "date_defeat": "August 15, 2022",
        "elected": "2017",
        "party": "R",
        "state": "Wyoming",
        "opponent": "Harriet Hageman",
    },
    ignore_index=True,
)

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_16622/940376627.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(


### clean up dates

In [18]:
df = df[df["date_defeat"].str.lower() != "date of defeat"]

In [19]:
df["clean_date"] = pd.to_datetime(df["date_defeat"].fillna(""))

In [20]:
df = df[
    (df["clean_date"] >= "2018-01-01") & (df["party"].str.strip() == "R")
].reset_index(drop=True)

### add ap state names

In [21]:
import us

In [22]:
df["ap_state"] = df["state"].map(us.states.mapping("name", "ap_abbr"))

### add column for office occupied and space for explainer
#### was gonna split the strings with their title, but they are all reps

In [23]:
df["explainer"] = ""

In [24]:
df["office"] = "House"

In [25]:
trump_endorse = [
    "Rep. David McKinley",
    "Rep. Tom Rice",
    "Rep. Peter Meijer",
    "Rep. Jaime Herrera Beutler",
    "Rep. Mark Sanford",
    "Rep. Liz Cheney",
]

In [26]:
df["trump"] = df["candidate"].isin(trump_endorse)

In [27]:
df["candidate"] = df["candidate"].str.replace("Rep. ", "", regex=False)

In [28]:
df["display_date"] = pd.to_datetime(df["clean_date"]).dt.strftime("%b. %Y")

In [29]:
df = df.drop(columns={"date_defeat"})

### upload clean and mostly complete dataframe to google sheets for graphics rig

In [30]:
wks_name = "new_data"
d2g.upload(
    df,
    "1ahhgRQiGOjUTxH2qBG1Cm3XDDAtZlHQDdhCniMSgr3k",
    wks_name,
    credentials=credentials,
    row_names=False,
)

<Worksheet 'new_data' id:218505413>